In [6]:
import os
import re
import json
import time
import requests
import pandas as pd
from ping3 import ping
from scapy.all import traceroute

In [12]:
def get_re_url(url, expr):
    result = {}
    
    response = requests.get(url)
    
    for k, v in expr.items():
        search_result = re.search(v, response.text)
        
        if search_result != None:
            result[k] = search_result.groups()[0]
        else:
            result[k] = ""
    
    return result

In [13]:
def request_extreme_ip_lookup():
    def wrapped(ip):
        data = get_re_url(f"https://extreme-ip-lookup.com/{ip}", {
            "data": 'URLIP={([^}]+)}'
        })

        obj = json.loads("{" + data["data"] + "}")
    
        return {
            "hostname": obj["ipName"]
            ,"country": obj["country"]
            ,"state": obj["region"]
            ,"city": obj["city"]
            ,"isp": obj["isp"]
            ,"latitude": obj["lat"]
            ,"longitude": obj["lon"]
        }
    
    return wrapped

In [14]:
def request_whatismyipaddress():
    def wrapped(ip):
        data = get_re_url(f"https://whatismyipaddress.com/ip/{ip}", {
            "hostname": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Hostname:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"country": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Country:[ |\t|\n||\r]*<\/th><td>(.*)<img'
            ,"state": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*State\/Region:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"city": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*City:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"isp": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*ISP:[ |\t|\n||\r]*<\/th><td>(.*)[ |\t|\n||\r]*<\/td>'
            ,"latitude": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Latitude:[ |\t|\n||\r]*<\/th><td>[ |\t|\n||\r]*(.*)[ |\t|\n||\r]*<\/td>'
            ,"longitude": '<tr>[ |\t|\n||\r]*<th>[ |\t|\n||\r]*Longitude:[ |\t|\n||\r]*<\/th><td>[ |\t|\n||\r]*(.*)[ |\t|\n||\r]*<\/td>'
        })
    
        return data
    
    return wrapped

In [32]:
def traceroute_table(uri, deltatime=1, minttl=1, maxttl=32, metainfo=False):
    result, _ = traceroute(uri, minttl=minttl, maxttl=maxttl, verbose=False)

    table = []
    for snd, rcv in result:
        tmp = (
            snd.ttl
            , rcv.src
            , (rcv.time - snd.sent_time)*1000
        )
        
        cols = ["ttl", "address", "ms"]
        
        if metainfo:
            result = request_extreme_ip_lookup()(rcv.src)
            time.sleep(deltatime)
            
            tmp = (
                *tmp
                , result["hostname"]
                , result["country"]
                , result["state"]
                , result["city"]
                , result["isp"]
                , result["latitude"]
                , result["longitude"]
            )
            
            cols = [*cols, "hostname", "country", "state", "city", "isp", "latitude", "longitude"]
        
        table.append(tmp)

    table_df = (
        pd.DataFrame(table, columns=cols)
        .drop_duplicates(subset=["address"], keep="first")
    )

    return table_df

In [34]:
traceroute_table("www.cmjornal.pt", metainfo=True)

,ttl,address,ms,hostname,country,state,city,isp,latitude,longitude
0,1,192.168.15.1,1.555681,,,,,,,
1,2,179.185.128.209,2.499342,179.185.128.209.dynamic.adsl.gvt.net.br,Brazil,Sao Paulo,Indaiatuba,Telefonica Brasil S.a,-23.08842,-47.2119
2,3,177.16.40.9,4.981756,177.16.40.9.static.host.gvt.net.br,Brazil,Parana,Curitiba,Telefonica Brasil S.a,-25.42778,-49.27306
3,4,152.255.133.70,4.060745,152-255-133-70.user.vivozap.com.br,Brazil,Sao Paulo,Sao Paulo,Telefonica Brasil S.a,-23.5475,-46.63611
4,6,216.184.112.148,16.278744,grtfortw1-0-1-0-24-0-4.net.telefonicaglobalsol...,United States,Florida,Miami,Telxius,25.77427,-80.19366
5,8,94.142.127.50,78.683853,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256
6,7,94.142.98.175,81.098795,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256
7,10,154.54.47.29,80.039978,be3401.ccr22.mia01.atlas.cogentco.com,United States,Florida,Miami,African Network Information Center,25.77427,-80.19366
8,12,154.54.24.221,95.787525,be2113.ccr42.dca01.atlas.cogentco.com,United States,Virginia,Arlington,African Network Information Center,38.88101,-77.10428
9,9,154.54.11.157,99.331617,be3017.ccr21.mia03.atlas.cogentco.com,United States,Florida,Miami,African Network Information Center,25.77427,-80.19366


In [12]:
traceroute_table("www.polytechnique.edu", metainfo=True)

,ttl,address,hostname,country,state,city,isp,latitude,longitude,ms
0,1,192.168.15.1,,,,,,,,0.719547
1,2,179.185.128.209,179.185.128.209.dynamic.adsl.gvt.net.br,Brazil,Sao Paulo,Indaiatuba,Telefonica Brasil S.a,-23.08842,-47.2119,2.481699
2,3,177.16.40.11,177.16.40.11.static.host.gvt.net.br,Brazil,Parana,Curitiba,Telefonica Brasil S.a,-25.42778,-49.27306,3.711224
3,4,152.255.133.70,152-255-133-70.user.vivozap.com.br,Brazil,Sao Paulo,Sao Paulo,Telefonica Brasil S.a,-23.5475,-46.63611,5.006313
4,6,5.53.1.202,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256,15.697002
5,7,94.142.127.50,,Spain,Madrid,Madrid,Telxius,40.4165,-3.70256,79.119682
6,8,84.16.15.165,,United States,New York,New York,Telxius,40.71427,-74.00597,82.249880
7,9,208.116.240.149,ae5.cr6-mia1.ip4.gtt.net,United States,Georgia,Atlanta,GTT,33.749,-84.38798,84.307909
8,10,213.200.119.214,et-3-3-0.cr4-par7.ip4.gtt.net,United States,Illinois,Chicago,GTT,41.85003,-87.65005,167.256832
10,11,193.51.180.131,,France,Nouvelle-Aquitaine,Pau,Renater,43.3,-0.36667,175.936460


In [7]:
LOL_NETWORK_LOGS_FOLDER = "C:\Riot Games\League of Legends\Logs\GameLogs"

logs_files_path = []

# Walk through the folders
for root, subfolders, files in os.walk(LOL_NETWORK_LOGS_FOLDER):
    # Pass by all files
    for file in files:
        # Filter the files with .csv extention
        if file.endswith(".csv"):
            # Add in the list
            logs_files_path.append(os.path.join(root, file))
            
            
ips = pd.DataFrame()


# Open and extract info
for log_path in logs_files_path:
    log = pd.read_csv(log_path, delimiter=",")[['network.rtt_average', 'network.rtt_jitter', 'network.peer_address', 'network.packets_lost']]
    
    # Transform the meas
    log['network.rtt_jitter'] = log['network.rtt_jitter'] * 1000
    log['network.rtt_average'] = log['network.rtt_average'] * 1000
    
    ips = pd.concat([ips, log])


# Group by
ips = ips.groupby('network.peer_address').mean().reset_index() \
         .rename(columns={'network.peer_address': 'network.ip', 'network.rtt_average': 'network.ping.ms'})


# Measure the new ping
ips['network.new_ping.ms'] = ips['network.ip'].apply(lambda x: ping(x) * 1000)

In [35]:
traces_df = pd.DataFrame()

for i, ip in enumerate(ips["network.ip"]):
    temp_df = traceroute_table(ip)
    temp_df["trace"] = i
    
    traces_df = pd.concat([traces_df, temp_df])

In [36]:
traces_df

,ttl,address,ms,trace
0,1,192.168.15.1,1.246214,0
1,2,179.185.128.209,4.538059,0
2,3,177.16.40.9,2.120256,0
3,4,152.255.140.69,61.657667,0
0,1,192.168.15.1,0.764608,1
1,2,179.185.128.209,1.924276,1
2,3,191.30.9.239,5.239010,1
3,4,152.255.140.69,64.937353,1
0,1,192.168.15.1,1.440525,2
1,2,179.185.128.209,4.419565,2
